### Load Data with PySpark

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

In [4]:
shows = spark.read.json("./data/shows-silicon-valley.json")

shows.show()
shows.printSchema()

+--------------------+--------------------+--------------------+--------+---+--------------------+--------+--------------+--------------------+--------------------+----------+------+-------+-----------------+------+--------------------+--------+----------+--------------------+----------+------+
|           _embedded|              _links|           externals|  genres| id|               image|language|          name|             network|        officialSite| premiered|rating|runtime|         schedule|status|             summary|    type|   updated|                 url|webChannel|weight|
+--------------------+--------------------+--------------------+--------+---+--------------------+--------+--------------+--------------------+--------------------+----------+------+-------+-----------------+------+--------------------+--------+----------+--------------------+----------+------+
|{[{{{http://api.t...|{{http://api.tvma...|{tt2575988, 27716...|[Comedy]|143|{http://static.tv...| English|Silic

## Breaking the second dimension

### Extracting elements from an array

In [5]:
array_subset = shows.select('name','genres')

array_subset.show()

+--------------+--------+
|          name|  genres|
+--------------+--------+
|Silicon Valley|[Comedy]|
+--------------+--------+



In [6]:
array_subset = array_subset.select(
    "name", 
    array_subset.genres[0].alias("dot_and_index"),
    F.col("genres")[0].alias("col_and_index"),
    array_subset.genres.getItem(0).alias("dot_and_method"),
    F.col("genres").getItem(0).alias("col_and_method"))

array_subset.show()

+--------------+-------------+-------------+--------------+--------------+
|          name|dot_and_index|col_and_index|dot_and_method|col_and_method|
+--------------+-------------+-------------+--------------+--------------+
|Silicon Valley|       Comedy|       Comedy|        Comedy|        Comedy|
+--------------+-------------+-------------+--------------+--------------+



### Performing multiple operations on an array column

In [7]:
array_subset_repeated = array_subset.select(
    "name",
    F.lit("Comedy").alias("one"),
    F.lit("Horror").alias("two"),
    F.lit("Drama").alias("three"),
    F.col("dot_and_index")
).select(
    "name",
    F.array("one","two","three").alias("Some_Genres"),
    F.array_repeat("dot_and_index", 5).alias("Repeated_Genres")
)

array_subset_repeated.show()

+--------------+--------------------+--------------------+
|          name|         Some_Genres|     Repeated_Genres|
+--------------+--------------------+--------------------+
|Silicon Valley|[Comedy, Horror, ...|[Comedy, Comedy, ...|
+--------------+--------------------+--------------------+



In [8]:
array_subset_repeated.select("name", F.size("Some_Genres"), F.size("Repeated_Genres")).show()

+--------------+-----------------+---------------------+
|          name|size(Some_Genres)|size(Repeated_Genres)|
+--------------+-----------------+---------------------+
|Silicon Valley|                3|                    5|
+--------------+-----------------+---------------------+



In [9]:
array_subset_repeated.select(
    "name",
    F.array_distinct("Some_Genres"),
    F.array_distinct("Repeated_Genres")
).show()

+--------------+---------------------------+-------------------------------+
|          name|array_distinct(Some_Genres)|array_distinct(Repeated_Genres)|
+--------------+---------------------------+-------------------------------+
|Silicon Valley|       [Comedy, Horror, ...|                       [Comedy]|
+--------------+---------------------------+-------------------------------+



In [10]:
array_subset_repeated = array_subset_repeated.select(
    "name",
    F.array_intersect("Some_Genres","Repeated_Genres").alias("Genres")
)

array_subset_repeated.show()

+--------------+--------+
|          name|  Genres|
+--------------+--------+
|Silicon Valley|[Comedy]|
+--------------+--------+



### Using ‘array_position()’ to search for ‘Genres’ string

In [11]:
array_subset_repeated.select(
    "Genres", F.array_position("Genres","Comedy")
).show()

+--------+------------------------------+
|  Genres|array_position(Genres, Comedy)|
+--------+------------------------------+
|[Comedy]|                             1|
+--------+------------------------------+



### The map type: keys and values within a column

In [12]:
columns = ['name', 'language', 'type']

shows_map = shows.select(
    *[F.lit(column) for column in columns],
    F.array(*columns).alias("values")
)

shows_map = shows_map.select(F.array(*columns).alias("keys"), "values")

shows_map.show(1)

+--------------------+--------------------+
|                keys|              values|
+--------------------+--------------------+
|[name, language, ...|[Silicon Valley, ...|
+--------------------+--------------------+



In [13]:
shows_map = shows_map.select(
    F.map_from_arrays("keys", "values").alias("mapped")
)

shows_map.printSchema()

root
 |-- mapped: map (nullable = false)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [14]:
shows_map.show(1, False)

+---------------------------------------------------------------+
|mapped                                                         |
+---------------------------------------------------------------+
|{name -> Silicon Valley, language -> English, type -> Scripted}|
+---------------------------------------------------------------+



In [15]:
shows_map.select(
    F.col("mapped.name"),
    F.col("mapped")["name"],
    shows_map.mapped["name"]
).show()

+--------------+--------------+--------------+
|          name|  mapped[name]|  mapped[name]|
+--------------+--------------+--------------+
|Silicon Valley|Silicon Valley|Silicon Valley|
+--------------+--------------+--------------+



## Nesting columns within columns 

In [16]:
shows.select('schedule').printSchema()

root
 |-- schedule: struct (nullable = true)
 |    |-- days: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- time: string (nullable = true)



In [17]:
shows_clean = shows.select(F.col('schedule.days'))
shows_clean.printSchema()

root
 |-- days: array (nullable = true)
 |    |-- element: string (containsNull = true)



## Building and using the data frame schema

In [22]:
import pyspark.sql.types as T

json_data = '{"name": "Alice", "age": 30, "city": "New York"}'

schema = T.StructType(
    [
        T.StructField("name", T.StringType()),
        T.StructField("age", T.IntegerType()),
        T.StructField("city", T.StringType()),
    ]
)

df_json = spark.createDataFrame([(json_data,)], ["json_data"]).select(F.from_json("json_data", schema).alias("data"))

df_json.printSchema()


root
 |-- data: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- age: integer (nullable = true)
 |    |-- city: string (nullable = true)



### Explode

In [28]:
json_data = '''
[
  {"name": "Hong", "grades": [{"subject": "Math", "score": 90}, {"subject": "English", "score": 88}]},
  {"name": "Lee", "grades": [{"subject": "Math", "score": 85}, {"subject": "English", "score": 92}]}
]
'''

df = spark.read.json(spark.sparkContext.parallelize([json_data]))

df.printSchema()

root
 |-- grades: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- score: long (nullable = true)
 |    |    |-- subject: string (nullable = true)
 |-- name: string (nullable = true)



In [29]:
df_exploded = df.select("name", F.explode("grades").alias("grade_info"))

df_exploded.show(truncate=False)

+----+-------------+
|name|grade_info   |
+----+-------------+
|Hong|{90, Math}   |
|Hong|{88, English}|
|Lee |{85, Math}   |
|Lee |{92, English}|
+----+-------------+



### Collect

In [30]:
df_collected = df.groupBy("name").agg(F.collect_list("grades").alias("collected_grades"))

df_collected.show(truncate=False)

+----+-----------------------------+
|name|collected_grades             |
+----+-----------------------------+
|Hong|[[{90, Math}, {88, English}]]|
|Lee |[[{85, Math}, {92, English}]]|
+----+-----------------------------+



### Struct

In [31]:
df_structured = df.select(
    F.col("name"),
    F.struct("grades.subject", "grades.score").alias("grades_struct")
)

df_structured.show(truncate=False)

+----+---------------------------+
|name|grades_struct              |
+----+---------------------------+
|Hong|{[Math, English], [90, 88]}|
|Lee |{[Math, English], [85, 92]}|
+----+---------------------------+

